## Cyberbullying Classification Project Using DistilBert Embeddings

Project Objective: To classify multiple types of cyberbullying
- Part 1: Ingest, Preprocess, Visualize, & Save (Processed) Dataset (Exploratory Data Analysis)
- Part 2: Train & Evaluate Modified Dataset

Dataset Source: https://www.kaggle.com/datasets/andrewmvd/cyberbullying-classification

##### Import Necessary Libraries

In [0]:
from sklearn.metrics import classification_report

from pyspark.sql.functions import *
import pyspark.sql.functions as F
from pyspark.sql.types import *
from pyspark.ml import Pipeline
from pyspark.sql.types import StructType, StructField, StringType

import sparknlp
from sparknlp.base import *
from sparknlp.annotator import *

##### Function to Ingest Dataset

In [0]:
def ingest_data(file: str, schema: StructType):
    file_type = "csv"
    infer_schema = "false"
    first_row_is_header = "true"
    delimiter = "\t"
    
    # The applied options are for CSV files. For other file types, these will be ignored.
    dataset = spark.read.format(file_type) \
      .option("inferSchema", infer_schema) \
      .option("header", first_row_is_header) \
      .option("sep", delimiter) \
      .schema(schema) \
      .load(file)
    
    dataset = dataset.na.drop()
    
    dataset = dataset.withColumn("input_word_length", F.size(F.split(F.col('text'), ' ')))
    
    return dataset

##### Ingest Dataset

In [0]:
file_location = "/FileStore/tables/cyberbullying_tweets.tsv"

orig_schema = StructType([
    StructField("text", StringType(), True),
    StructField("label", StringType(), True)
])

df = ingest_data(file_location, orig_schema)

display(df)

##### How Many Samples in Dataset?

In [0]:
df.count()

Out[75]: 47678

##### Split Dataset into Train/Test Datasets (80/20)

In [0]:
print("Total Number of Samples in Dataset:", df.count())

train_ds, test_ds = df.randomSplit(weights=[0.80,0.20], seed=42)

print("# of Training Samples:", train_ds.count())
print("# of Testing Samples:", test_ds.count())

Total Number of Samples in Dataset: 47678
# of Training Samples: 38225
# of Testing Samples: 9453


##### Basic Values/Constants

In [0]:
NUM_OF_EPOCHS = 12
BATCH_SIZE = 64
MAX_INPUT_LEN = 63
LR = 5e-3
VERBOSITY_LEVEL = 1

##### Instantiate Pipeline Stages & Build Pipeline

In [0]:
# Pipeline Stages
doc_assembler = DocumentAssembler() \
    .setInputCol("text") \
    .setOutputCol("document")

use = UniversalSentenceEncoder.pretrained('tfhub_use', lang="en") \
    .setInputCols(["document"])\
    .setOutputCol("sent_embeds")

# clf_model
clf = ClassifierDLApproach()\
    .setInputCols(["sent_embeds"])\
    .setOutputCol("class")\
    .setLabelColumn("label")\
    .setBatchSize(BATCH_SIZE)\
    .setLr(LR)\
    .setMaxEpochs(NUM_OF_EPOCHS)\
    .setVerbose(VERBOSITY_LEVEL)


tfhub_use download started this may take some time.
Approximate size to download 923.7 MB
[OK!]


##### Build Pipeline

In [0]:
cyber_clf_pipe = Pipeline(stages=[doc_assembler, use, clf])

##### Fit Pipeline to Training Dataset to Create Model

In [0]:
cyber_clf_model = cyber_clf_pipe.fit(train_ds)

##### Make & Display Predictions Using Test Dataset

In [0]:
preds = cyber_clf_model.transform(test_ds)
display(preds)

text label input_word_length document sent_embeds class Keeks is a bitch she curves everyone lol I walked into a conversation like this. Smh gender 18 List(List(document, 0, 85, Keeks is a bitch she curves everyone lol I walked into a conversation like this. Smh, Map(sentence -> 0), List())) List(List(sentence_embeddings, 0, 85, Keeks is a bitch she curves everyone lol I walked into a conversation like this. Smh, Map(sentence -> 0, token -> Keeks is a bitch she curves everyone lol I walked into a conversation like this. Smh, pieceId -> -1, isWordStart -> true), List(-0.014596484, 0.007908661, -0.0106495945, -0.02620722, 0.014220874, -0.040270634, 0.013564525, -0.018352883, -0.024856647, 0.027991798, 0.031696703, -0.033065513, 0.04887861, -0.038917582, -0.013669127, 0.05427907, 8.3889003E-4, -0.050969135, -0.033709303, 0.09998081, -0.06825616, -0.04985333, 0.020699244, 0.021139069, 0.043930933, -0.039650943, -0.0034008457, -0.059185773, 0.08451999, -0.0053838985, -0.06064561, 0.042419456, 0.06170354, -0.060266405, 0.047422934, -0.015039042, -0.07109326, -0.020179268, 0.0151972575, -0.025202708, -0.084316626, 0.028414808, 0.03560471, -0.061057627, -0.057022266, 0.047245547, -0.015998736, 0.05036492, 0.0247513, 0.008466148, -0.015609272, 0.04700374, 0.015510199, -0.0566997, 0.059715986, 0.04562695, 0.018701863, -0.0017216008, 0.027126102, 0.016910542, 0.022939326, -0.102326535, -0.04117303, 0.010898724, -0.014242792, -0.001871676, 0.023545511, 0.017547453, 0.08158247, 0.04556215, -0.03582235, 0.027543325, -0.06251386, -0.044929575, -0.048410214, 7.1596995E-4, 0.053470373, 0.040847033, -0.039021105, 0.04129036, 0.05459041, 0.052486733, 0.081221625, 0.08263451, 0.016606955, -0.0170772, -0.007304159, 0.008094125, 0.07423002, -0.052839946, 0.06476109, 0.065057464, -0.0051375013, 0.043111242, -0.028959448, 0.010368012, 0.03269065, -0.0020537786, 0.052616917, -0.017586764, 0.03854689, -0.042943772, 0.026313799, 0.043411538, -0.010063493, 0.025858333, -0.030199638, -0.044633202, -0.07930968, 0.040631413, -0.05978511, 0.050591838, -0.014253196, 0.022812279, -0.057749514, 0.029527025, 0.0073533934, -0.06347939, 0.06690878, 0.046065908, -0.017120248, 0.045476347, 0.0041423077, 0.03273353, -0.01287591, -0.026897773, 0.0057481686, -0.015300496, 0.022362849, -0.016584441, -0.063571595, 0.06925198, 0.002120998, 0.03308181, 0.05987348, 0.070357196, -0.03707492, 0.020634588, 0.06659026, -0.038478393, -0.021450592, 0.032427117, -0.03798794, 0.014767486, -0.012219482, 0.017835997, 0.048997596, -0.019784283, -0.023602277, -0.047096133, -0.014295271, -0.052924585, 0.053940848, 3.4939905E-4, 0.06799483, 0.016453652, -0.07910609, -0.031542134, -0.041261293, 0.006922343, -0.056305397, -0.0017010998, 0.08472985, -0.030366004, -0.020253908, -0.006738786, -0.05261635, 0.040110547, 0.055785198, 8.330419E-4, -0.004751366, -0.03554917, -0.04879162, 0.0551674, -0.0033659448, 0.045921348, 0.05006836, 0.0065674316, -0.068996646, -0.018633502, 0.037732605, -0.020195499, 0.0046943775, 0.030719273, -0.061170746, -0.031589482, 0.066158555, -0.034206487, -0.03609865, -0.0025720748, -0.009588671, -0.03392314, -0.04401867, -0.013975706, -0.03191251, -0.048131436, 0.0062650004, -0.015955871, 0.015476881, 0.061410304, 0.02088843, -0.022481238, 0.016215483, 0.013816231, 0.08268097, 0.07333676, 0.0613737, -0.0047365907, 0.012288539, -0.04669661, 0.062587135, -0.04596741, -0.039304823, 0.046222385, 0.001016779, 0.085381545, 0.013646839, 0.023263711, 0.02832825, 0.027854802, -0.026563765, -0.010763361, 0.0311306, -0.041549556, -0.042048987, 0.05979649, -0.089504905, 0.08189701, -0.039959297, -0.06777887, 0.034765583, 0.07663458, 0.033156738, -0.0429542, 0.020488534, -0.09989961, 0.08121526, -0.02659475, 0.01368942, 0.040366795, -0.03437976, 0.017565101, 0.004178028, 0.0050219586, 0.0017380012, -0.056922607, -0.017630707, 0.022214517, -0.04049047, 0.06808544, 0.045661837, -0.028689718, 0.0012308746, 0.06420778, 0.0044980054, 0.09512474, 0.046650868, -0.033085782, 0.006279162

##### Convert Necessary Features/Classes to Pandas DataFrame

In [0]:
preds_in_pandas = (preds.select(F.col('text').alias("text"), F.col('label').alias("ground_truth"), F.col('class.result').alias("prediction"))).toPandas()
display(preds_in_pandas)

text,ground_truth,prediction
Keeks is a bitch she curves everyone lol I walked into a conversation like this. Smh,gender,List(gender)
bitch get up off me,gender,List(not_cyberbullying)
bitch who do you love,gender,List(gender)
i need a trippy bitch who fuck on Hennessy,gender,List(other_cyberbullying)
lames crying over hoes thats tears of a clown,gender,List(gender)
young Pill Chamberlain these bitches love my music,gender,List(not_cyberbullying)
!!!!!!!!!!!!! RT @ShenikaRoberts: The shit you hear about me might be true or it might be faker than the bitch who told it to ya ,other_cyberbullying,List(gender)
"""#50factsaboutme I always feel like my """"friends"""" just pretend to like me because they only talk to me in school""",not_cyberbullying,List(age)
"""#MajorTurnOffs tasteless jokes about cutting/suicide/gay people/rape/the holocaust/dead babies/""""retards"""". Ugh. So not okay.""",gender,List(gender)
"""#MarriageEquality for """"Gay Marriage"""" Is a Joke. Is Rape Equality Next? Pedophilia Equality? Bestiality Equality? #KimDavis Is Right #Truth""",gender,List(gender)


##### Create & Display Classification Report

In [0]:
preds_in_pandas['prediction'] = preds_in_pandas['prediction'].apply(lambda x : x[0])

report = classification_report(preds_in_pandas['ground_truth'], preds_in_pandas['prediction'])
print(report)

                     precision    recall  f1-score   support

                age       0.94      0.96      0.95      1589
          ethnicity       0.92      0.94      0.93      1566
             gender       0.85      0.84      0.84      1581
  not_cyberbullying       0.66      0.50      0.57      1597
other_cyberbullying       0.60      0.71      0.65      1527
           religion       0.91      0.95      0.93      1593

           accuracy                           0.82      9453
          macro avg       0.81      0.82      0.81      9453
       weighted avg       0.82      0.82      0.81      9453



##### End/Stop Spark Session

In [0]:
#spark.stop()